In [ ]:
!pip install scikit-image
!pip install seaborn
!pip install torch torchvision

In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet34

from datasets import MelSpecDataset

In [2]:
model = resnet34()

In [3]:
BATCH_SIZE = 2
TEST_SIZE = 0.2

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset = MelSpecDataset(csv_loc="./data/wav_files.csv", data_dir="./data")

# train-test split
test_index = int(len(dataset)*TEST_SIZE)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-test_index]) # first 20%
dataset_test = torch.utils.data.Subset(dataset, indices[-test_index:]) # last 80%

# define data loader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


In [4]:
#def main():

# get model
model = resnet34()
model.to(device)

# construct optimiser
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss()

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)


In [5]:
# train for num_epochs
num_epochs = 1

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    running_loss = 0.0
    
    for i, data in enumerate(data_loader):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataset.py", line 257, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/storage/music-genre-classification/datasets.py", line 28, in __getitem__
    spec = io.imread(spec_path)
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/skimage/io/_io.py", line 48, in imread
    img = cal

In [6]:
next(iter(data_loader))

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataset.py", line 257, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/storage/music-genre-classification/datasets.py", line 28, in __getitem__
    spec = io.imread(spec_path)
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/skimage/io/_io.py", line 48, in imread
    img = cal